In [7]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import joblib

# --- CONFIGURAÇÕES ---
# Nome do arquivo onde o modelo será salvo
ARQUIVO_MODELO = 'sentiment_model.pkl'

# URL do dataset (usando o link raw correto da sua branch ou o público)
url_dados = 'https://raw.githubusercontent.com/Hackaton-ONE/hackathon-DataScience/refs/heads/feature/mvp-sentiment/olist_order_reviews_dataset.csv'

print("⏳ Baixando dados da nuvem...")
try:
    df = pd.read_csv(url_dados)
    print("✅ Dados carregados com sucesso!")
except Exception as e:
    # Fallback para o link público caso o link privado falhe
    print("⚠️ Falha no link do repo. Tentando link público...")
    url_publica = "https://raw.githubusercontent.com/BrunnerS/Olist_Dataset/master/olist_order_reviews_dataset.csv"
    df = pd.read_csv(url_publica)
    print("✅ Dados carregados do link público!")

# 1. Download de recursos de texto (silencioso)
nltk.download('stopwords', quiet=True)
stop_words_pt = stopwords.words('portuguese')

# --- FUNÇÕES ---

def limpar_texto_final(texto):
    """Função de produção para limpeza de texto."""
    if not isinstance(texto, str):
        return ""

    texto = texto.lower()
    # Remove pontuação e números, mantém acentos
    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ ]', '', texto)

    # Remove stopwords
    palavras = [p for p in texto.split() if p not in stop_words_pt]
    return " ".join(palavras)

def classificar_sentimento(nota):
    """Regra de Negócio: 1-2 (Neg), 3 (Neu), 4-5 (Pos)"""
    if nota <= 2:
        return 'Negativo'
    elif nota == 3:
        return 'Neutro'
    else:
        return 'Positivo'

# --- PIPELINE DE EXECUÇÃO ---

print("\n🚀 Iniciando Pipeline de Processamento...")

# Limpeza de nulos antes de processar
df = df.dropna(subset=['review_comment_message', 'review_score'])

# 2. Pré-processamento
print("⚙️ Processando textos e sentimentos...")
df['sentiment'] = df['review_score'].apply(classificar_sentimento)
df['text_clean'] = df['review_comment_message'].apply(limpar_texto_final)

# 3. Divisão Treino/Teste
X = df['text_clean']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Construção do Modelo (Pipeline)
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=2000)),
    ('clf', LogisticRegression(solver='liblinear', class_weight='balanced'))
])

# 5. Treinamento
print("🧠 Treinando o modelo (pode levar alguns segundos)...")
pipeline.fit(X_train, y_train)

# 6. Validação
y_pred = pipeline.predict(X_test)
acuracia = accuracy_score(y_test, y_pred)

print(f"\n✅ Modelo Treinado com Sucesso!")
print(f"📊 Acurácia do Modelo: {acuracia:.2f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# 7. Exportação
joblib.dump(pipeline, ARQUIVO_MODELO)
print(f"\n💾 Arquivo '{ARQUIVO_MODELO}' salvo no Colab. Faça o download dele agora!")

⏳ Baixando dados da nuvem...
✅ Dados carregados com sucesso!

🚀 Iniciando Pipeline de Processamento...
⚙️ Processando textos e sentimentos...
🧠 Treinando o modelo (pode levar alguns segundos)...

✅ Modelo Treinado com Sucesso!
📊 Acurácia do Modelo: 0.82

Relatório de Classificação:
              precision    recall  f1-score   support

    Negativo       0.76      0.83      0.79      2151
      Neutro       0.27      0.24      0.25       730
    Positivo       0.92      0.89      0.91      5315

    accuracy                           0.82      8196
   macro avg       0.65      0.66      0.65      8196
weighted avg       0.82      0.82      0.82      8196


💾 Arquivo 'sentiment_model.pkl' salvo no Colab. Faça o download dele agora!
